In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

In [ ]:
ip_link = ''

ip = pd.read_excel(ip_link, sheet_name='ValueWeigtedReturns').dropna(how='all')

ip = ip.set_index('Unnamed: 0')
ip.index.name = None

# map from first to last day of month
ip.index = ip.index.map(lambda x: x + pd.tseries.offsets.MonthEnd(0))

ip = ip / 100

ip.head()

In [ ]:
# read data
factors = pd.read_excel('f_capW-quintiles.xlsx', index_col=0)

# missing information
factors = factors.dropna(how='any')

factors.head()

In [ ]:
(ip + 1).loc[ip.index > '01-01-1990'].cumprod().plot(logy=True, legend=True)

In [ ]:
(factors[factors.columns[:7]] + 1).cumprod().plot(logy=True, legend=True)



In [ ]:
model_spec = [
    ['Mkt-RF', 'SMB', 'HML', 'MOM'],
    ['SUS_soc', 'SUS_env', 'SUS_gov', 'Mkt-RF', 'SMB', 'HML', 'MOM']
]

nw_lags = 12

ip_cols = ip.columns
factors = factors.join(ip)

for col in ip_cols:
    factors[col] = factors[col] - factors['RF']

factors.head()

In [ ]:
def estimate_model(df, capm_fac, indu_pf, name):
    estimates = pd.DataFrame()

    for industry in indu_pf:
        X = sm.add_constant(df[capm_fac])
        X = X.rename(columns={'const': 'alpha'})
        Y = df[industry]

        model = sm.OLS(endog=Y, exog=X)
        model = model.fit(cov_type='HAC', cov_kwds={'maxlags': nw_lags})

        # save parameters
        parameters = model.params
        parameters.name = 'Estimates'

        parameters = pd.DataFrame(parameters)

        # save t statistics
        tvalues = model.tvalues
        tvalues['Mkt-RF'] = (model.params['Mkt-RF'] - 1) / model.bse['Mkt-RF']
        tvalues.name = 'T'
        parameters = parameters.join(tvalues)

        # save std. errors
        std = model.bse
        std.name = 'std. error'
        parameters = parameters.join(std)

        # save R^2
        parameters['R2'] = model.rsquared_adj

        # save name
        parameters['Portfolio'] = industry

        estimates = estimates.append(parameters)

    estimates['Type'] = name

    return estimates


ip_estimates = estimate_model(factors, model_spec[0], ip_cols, 'Carhart')
ip_estimates = ip_estimates.append(estimate_model(factors, model_spec[1], ip_cols, 'Sustainable'))

ip_estimates

In [ ]:
# adding factors one by one
corr_test = estimate_model(factors, ['Mkt-RF', 'SMB', 'HML', 'MOM', 'SUS_soc'], ip_cols, 'Social')
corr_test = corr_test.append(estimate_model(factors, ['Mkt-RF', 'SMB', 'HML', 'MOM', 'SUS_gov'], ip_cols, 'Governance'))
corr_test = corr_test.append(estimate_model(factors, ['Mkt-RF', 'SMB', 'HML', 'MOM', 'SUS_env'], ip_cols, 'Environment'))

corr_test = corr_test.append(estimate_model(factors, model_spec[1], ip_cols, 'S-CAPM'))
corr_test['Parameter'] = corr_test.index
corr_test = corr_test.reset_index(drop=True)
corr_test.head()

In [ ]:
corr_test.loc[(corr_test['Parameter'] == 'alpha') & (corr_test['Type'].isin(['S-CAPM']))]['Estimates']

In [ ]:
fig, axes = plt.subplots(3, 1, figsize = (5, 10), sharey=False)

axes[0].scatter(y=corr_test.loc[(corr_test['Parameter'] == 'SUS_env') & (corr_test['Type'].isin(['Environment']))]['Estimates'], 
                x=corr_test.loc[(corr_test['Parameter'] == 'SUS_env') & (corr_test['Type'].isin(['S-CAPM']))]['Estimates'])

axes[1].scatter(y=corr_test.loc[(corr_test['Parameter'] == 'SUS_soc') & (corr_test['Type'].isin(['Social']))]['Estimates'], 
                x=corr_test.loc[(corr_test['Parameter'] == 'SUS_soc') & (corr_test['Type'].isin(['S-CAPM']))]['Estimates'])

axes[2].scatter(y=corr_test.loc[(corr_test['Parameter'] == 'SUS_gov') & (corr_test['Type'].isin(['Governance']))]['Estimates'], 
                x=corr_test.loc[(corr_test['Parameter'] == 'SUS_gov') & (corr_test['Type'].isin(['S-CAPM']))]['Estimates'])


axes[0].set_xlabel(r'S-CAPM $\left(\widehat{e}\right)$')
axes[1].set_xlabel(r'S-CAPM $\left(\widehat{c}\right)$')
axes[2].set_xlabel(r'S-CAPM $\left(\widehat{g}\right)$')

axes[0].set_ylabel(r'Reduced model $\left(\widehat{e}\right)$')
axes[1].set_ylabel(r'Reduced model $\left(\widehat{c}\right)$')
axes[2].set_ylabel(r'Reduced model $\left(\widehat{g}\right)$')

plt.tight_layout()
plt.savefig('reducedmodel.png', format='png', dpi=1000)
plt.show()

In [ ]:
car = ip_estimates.loc[ip_estimates['Type'] == 'Carhart']

car_results = pd.DataFrame(index=[
    'alpha', 'Mkt-RF', 'SMB', 'HML', 'MOM', 'alpha (t)', 'Mkt-RF (t)', 'SMB (t)', 'HML (t)', 'MOM (t)', 'Adj R2'])

for subdf in car.groupby(by='Portfolio'):
    s = subdf[1][['Estimates']]
    
    t = subdf[1][['T']]
    t.index = t.index.map(lambda x: f'{x} (t)')
    t = t.rename(columns={'T': 'Estimates'})

    r2 = pd.Series(subdf[1]['R2'].unique()[0])
    r2.name = 'Estimates'
    r2.index = ['Adj R2']
    r2 = pd.DataFrame(r2)

    s = s.append(t)
    s = s.append(r2)
    s = s.rename(columns={'Estimates': subdf[0]})

    car_results = car_results.join(s)

car_results.sort_index(ascending=True)

In [ ]:
print(car_results.to_latex(float_format='%.4f'))

In [ ]:
sus = ip_estimates.loc[ip_estimates['Type'] == 'Sustainable']

sus_results = pd.DataFrame(index=[
    'alpha', 'Mkt-RF', 'SMB', 'HML', 'MOM', 'SUS_gov', 'SUS_env', 'SUS_soc', 'SUS_soc (t)', 'SUS_gov (t)', 'SUS_env (t)', 'alpha (t)', 'Mkt-RF (t)', 'SMB (t)', 'HML (t)', 'MOM (t)', 'Adj R2'])

for subdf in sus.groupby(by='Portfolio'):
    s = subdf[1][['Estimates']]
    
    t = subdf[1][['T']]
    t.index = t.index.map(lambda x: f'{x} (t)')
    t = t.rename(columns={'T': 'Estimates'})

    r2 = pd.Series(subdf[1]['R2'].unique()[0])
    r2.name = 'Estimates'
    r2.index = ['Adj R2']
    r2 = pd.DataFrame(r2)

    s = s.append(t)
    s = s.append(r2)
    s = s.rename(columns={'Estimates': subdf[0]})

    sus_results = sus_results.join(s)

sus_results.sort_index(ascending=True)

In [ ]:
print(sus_results.to_latex(float_format='%.4f'))

In [ ]:
def calc_GRS(F, e, a):
    ddof = 0  # Delta degrees of freedom set to 0 to align with MatLab
    k = 0  # number of factors
    N = e.shape[1]  # number of assets 
    T = e.shape[0]  # time of time-series observations
    
    a = np.matrix(a['alpha'].values)  # estimated alphas (1 x N)
    sigma = e.cov(ddof=ddof)  # residual covariance (N x N)
    mu = np.matrix(F.mean().values)  # k x 1 vector of factor means
    
    if len(F.shape) == 1:
        omega = F.var(ddof=ddof)  # 1 x 1 vector of factor variances
        k = 1
    else:
        omega = F.cov(ddof=ddof)  # k x k covariance matix for the k factors
        k = F.shape[1]
    
    # calculating GRS
    GRS = ((T-N-k)/N) * np.dot(np.linalg.inv((1 + mu * np.linalg.inv(omega) * mu.T)), a * np.linalg.inv(sigma) * a.T)
    p_val = 1 - stats.f.cdf(GRS, N, (T-N-k))
    
    return {
        'GRS': GRS.item(),
        'P value': p_val.item(),
        'Avg. absolute alpha': np.mean(np.abs(a)),
        'T': T,
        'N': N,
        'k': k
    }


for model in model_spec:
    resids = pd.DataFrame()
    results = pd.DataFrame()
    
    for asset in ip_cols:
        X = factors[model]
        X = sm.add_constant(X)
        Y = factors[asset]

        fit = sm.OLS(endog=Y, exog=X).fit(cov_type='HAC', cov_kwds={'maxlags': nw_lags})

        fit.params = fit.params.rename({'const': 'alpha'})
        results = results.append(fit.params, ignore_index=True)

        resids[asset] = fit.resid.values

    display(calc_GRS(F=factors[model], e=resids, a=results))

In [ ]:
t_coef_up = stats.t.ppf(1-0.05, 150)
t_coef_low = stats.t.ppf(0.05, 150)

def range_of(s):
    return round(s.min(), 2)

model_compare = {
    'Carhart': {
        'Range of Adj. R2': f"{range_of(car_results.loc['Adj R2'])}-{range_of(car_results.loc['Adj R2'].max())}",
        'Mean Adj. R2': round(car_results.loc['Adj R2'].mean(), 4),
        'a': f"{(car_results.loc['alpha (t)'] > t_coef_up).sum()} ({(car_results.loc['alpha (t)'] < t_coef_low).sum()})",
        'b': f"{(car_results.loc['Mkt-RF (t)'] > t_coef_up).sum()} ({(car_results.loc['Mkt-RF (t)'] < t_coef_low).sum()})",
        'h': f"{(car_results.loc['HML (t)'] > t_coef_up).sum()} ({(car_results.loc['HML (t)'] < t_coef_low).sum()})",
        's': f"{(car_results.loc['SMB (t)'] > t_coef_up).sum()} ({(car_results.loc['SMB (t)'] < t_coef_low).sum()})",
        'm': f"{(car_results.loc['MOM (t)'] > t_coef_up).sum()} ({(car_results.loc['MOM (t)'] < t_coef_low).sum()})"
        },
    'SCAPM': {
        'Range of Adj. R2': f"{range_of(sus_results.loc['Adj R2'])}-{range_of(sus_results.loc['Adj R2'].max())}",
        'Mean Adj. R2': round(sus_results.loc['Adj R2'].mean(), 4),
        'a': f"{(sus_results.loc['alpha (t)'] > t_coef_up).sum()} ({(sus_results.loc['alpha (t)'] < t_coef_low).sum()})",
        'b': f"{(sus_results.loc['Mkt-RF (t)'] > t_coef_up).sum()} ({(sus_results.loc['Mkt-RF (t)'] < t_coef_low).sum()})",
        'h': f"{(sus_results.loc['HML (t)'] > t_coef_up).sum()} ({(sus_results.loc['HML (t)'] < t_coef_low).sum()})",
        's': f"{(sus_results.loc['SMB (t)'] > t_coef_up).sum()} ({(sus_results.loc['SMB (t)'] < t_coef_low).sum()})",
        'm': f"{(sus_results.loc['MOM (t)'] > t_coef_up).sum()} ({(sus_results.loc['MOM (t)'] < t_coef_low).sum()})",

        'g': f"{(sus_results.loc['SUS_gov (t)'] > t_coef_up).sum()} ({(sus_results.loc['SUS_gov (t)'] < t_coef_low).sum()})",

        'e': f"{(sus_results.loc['SUS_env (t)'] > t_coef_up).sum()} ({(sus_results.loc['SUS_env (t)'] < t_coef_low).sum()})",

        'c': f"{(sus_results.loc['SUS_soc (t)'] > t_coef_up).sum()} ({(sus_results.loc['SUS_soc (t)'] < t_coef_low).sum()})"
        },
}

sig = pd.DataFrame(model_compare).transpose().fillna(0)
sig

In [ ]:
### FAMA MACBETH ###

fac = factors[model_spec[1]]
sv = factors[ip.columns]

# First-pass time-series regressions
resids = pd.DataFrame()
results = pd.DataFrame()

for asset in sv.columns:
    Y = sv[asset]
    X = fac
    X = sm.add_constant(X)
    
    fit = sm.OLS(endog=Y, exog=X).fit(cov_type='HAC', cov_kwds={'maxlags': nw_lags})
    
    results = results.append(fit.params, ignore_index=True)  # save betas
    resids = resids.append(fit.resid, ignore_index=True)


results.index = sv.columns
results = results.drop(columns='const')

resids.index = sv.columns
resids = resids.transpose()


# Second-stage regressions
lambda_alpha_t = pd.DataFrame()
r2_t = []

for t in range(fac.shape[0]):
    X = results
    X = sm.add_constant(X)
    
    Y = sv.iloc[t]
    
    fit = sm.OLS(endog=Y, exog=X).fit(cov_type='HAC', cov_kwds={'maxlags': 1})
    
    lambda_alpha_t = lambda_alpha_t.append(fit.params, ignore_index=True)
    r2_t.append(fit.rsquared_adj)

lambda_alpha_t = lambda_alpha_t.rename(columns={'const': 'c'})


def FM_t_stat(T, estimates_vector, hat_bar):
    var_val = 0
    
    for t in range(T):
        var_val += np.power((estimates_vector.iloc[t] - hat_bar), 2)
    
    var_val = var_val * (1 / (T-1))
    
    se_val = np.sqrt(var_val) / np.sqrt(T)
    
    t_stat = hat_bar / se_val
    
    p_val = 1 - stats.norm.cdf(np.abs(t_stat))  # asym. normally distributed
    
    return {
        'Parameter': estimates_vector.name,
        'Estimate': hat_bar,
        'FM t-stat': t_stat,
        'P-value': p_val
    }
    

res = []
for parameter in lambda_alpha_t.columns:
    res.append(FM_t_stat(lambda_alpha_t.shape[0], lambda_alpha_t[parameter], lambda_alpha_t[parameter].mean()))



res = pd.DataFrame(res)
res['Adj. R2'] = np.mean(r2_t)
print(res.to_latex(float_format='%.4f', index=False))

In [ ]:
results

In [ ]:
# calculating R2
X = results
X = sm.add_constant(X)

Y = sv.mean()

fit = sm.OLS(endog=Y, exog=X).fit(cov_type='HAC', cov_kwds={'maxlags': 1})
print(f'Cross sectional R2: {fit.rsquared_adj}')

In [ ]:
# calculating predicted returns
intercept = lambda_alpha_t['c'].mean()
intercept = np.matrix(intercept)

lambdas = lambda_alpha_t.drop(columns='c').mean().values
lambdas = np.matrix(lambdas)

betas = results.values
betas = np.matrix(betas)

# runnding prediction
pred_returns = intercept + np.dot(betas, lambdas.T)


plt.scatter(sv.mean().values, np.array(pred_returns))
plt.xlabel('Realized returns')
plt.ylabel('Predicted returns')
plt.title('SCAPM, $R^2=0.89$')
plt.show()

In [ ]:
pred_returns_carhart  # 0.51
pred_returns_scapm  # 0.89

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (12,4), sharey=True)


axes[0].scatter(sv.mean().values, np.array(pred_returns_carhart))
axes[1].scatter(sv.mean().values, np.array(pred_returns_scapm))
axes[0].set_xlabel('Realized returns')
axes[1].set_xlabel('Realized returns')
axes[0].set_ylabel('Predicted returns')

vals = axes[0].get_yticks()
axes[0].set_yticklabels(['{:,.2%}'.format(x) for x in vals])

vals = axes[0].get_xticks()
axes[0].set_xticklabels(['{:,.2%}'.format(x) for x in vals])

vals = axes[1].get_xticks()
axes[1].set_xticklabels(['{:,.2%}'.format(x) for x in vals])

axes[0].set_title('Carhart, $R^2_{adj.}=0.51$')
axes[1].set_title('S-CAPM, $R^2_{adj.}=0.89$')

plt.savefig('famamacbeth.png', format='png', dpi=300)

plt.show()